!! TO DO: understand math behind good cases ratio w.r.t. std/scale_up_factor (currently hyps tuned based on experimental evidence) !!






In [1]:
import numpy as np
import pandas as pd
import random
from scipy.special import softmax
from statistics import mean
from tqdm import tqdm

# hyperparameters
no_puzzlepieces = 15
no_instances = 2000000
std = 4#2.75   #0.275 # noise in coordinate confidence                           
std_inst = 0.075 # noise in instruction confidence         
coord = list(range(no_puzzlepieces))
scale_up_factor = 13#10   #1
scale_up_factor_inst = 3 # parameter to scale up all values before softmaxing to prevent all values from being all too similar

# Instructions and Instruction Confidence

In [2]:
X_inst_conf_ideal =[]
X_inst_conf_noisy =[]

for m in tqdm(range(no_instances)):
        
    #print('\n')
    #print(f'instance: {m}')
    
    inst_p_ideal = []
    inst_p_noisy = []
    
    for group in range(3):
        
        #print('\n')
        #print(f'group: {group}')
        
        #############################################################################
        ############## SET TRUE INSTRUCTION CONFIDENCE ##############################
        #############################################################################
        
        current_inst_p_ideal = random.randint(0,1)
        #print(current_inst_p_ideal)
        inst_p_ideal.append(current_inst_p_ideal)
        
        
        #############################################################################
        ########################  NORMALLY DISTRIBUTED NOISE ########################
        #############################################################################
        
        # create an auxilary probability that an instruction was not given (1 - inst_p) 
        inst_p_ideal_aux = []
        inst_p_ideal_aux.append(current_inst_p_ideal) # probability that an instruction was given
        if current_inst_p_ideal == 0: # probability that an instruction was not given
            inst_p_ideal_aux.append(1) 
        else:
            inst_p_ideal_aux.append(0) 

        
        # upscale values
        inst_p_aux_upscaled = np.multiply(inst_p_ideal_aux, scale_up_factor_inst) 
        #print(inst_p_aux_upscaled) 
        
        # noise up those ideal probabilities (n) for "instruction was given" and "instruction was not given"
        inst_p_noisy_aux = []
        for n in range(2):
            inst_p_noisy_aux.append(np.random.normal(scale = std_inst, loc = inst_p_aux_upscaled[n], size = 1)[0])                             
        #print(inst_p_noisy_aux)

        # squash values to sum up to one to represent probabilities
        inst_p_noisy_aux_squashed = softmax(inst_p_noisy_aux)
        #print(inst_p_noisy_aux_squashed)
        
        #print(inst_p_noisy_aux_squashed[0])
        inst_p_noisy.append(inst_p_noisy_aux_squashed[0])
    
    # store
    X_inst_conf_ideal.append(inst_p_ideal)
    X_inst_conf_noisy.append(inst_p_noisy)


100%|██████████| 2000000/2000000 [07:00<00:00, 4754.65it/s]


# Coordinates and Coordinate Confidence

In [3]:
X_coord_conf_ideal =[]
X_coord_conf_noisy =[]

for m in tqdm(range(no_instances)):
    
    
    #print('\n')
    #print(f'instance: {m}')
    

    #############################################################################
    ############## SET TRUE COORDINATE & IDEAL PROBABILITIES ####################
    #############################################################################


    # set all coordinate probabilities p = 0.0
    coordinate_p_ideal = np.zeros(no_puzzlepieces)

    # determine true coordinate and set its probability to p = 1.0
    i = random.randint(0,no_puzzlepieces-1)
    coordinate_p_ideal[i] = 1.0

    # show all probabilities
    #print(coordinate_p_ideal)
    
    # store ideal probabilities
    X_coord_conf_ideal.append(coordinate_p_ideal)


    #############################################################################
    ########################  NORMALLY DISTRIBUTED NOISE ########################
    #############################################################################
    
    # upscale values
    coordinate_p_ideal_upscaled = np.multiply(coordinate_p_ideal, scale_up_factor) 
    
    # auxilary lists to store different group's outputs per instance
    both_groups = []
    for group in range(2):
        
        if X_inst_conf_ideal[m][group]==0:
            
            # add uniformly distributed noise on ideal probabilities 
            coordinate_p_noisy = []
            for i in range(no_puzzlepieces):
                coordinate_p_noisy.append(np.random.uniform(0.0, scale_up_factor/scale_up_factor_inst))
            
        else:    

            # add normally distributed noise on ideal probabilities 
            coordinate_p_noisy = []
            for i in range(no_puzzlepieces):
                coordinate_p_noisy.append(np.random.normal(scale = std, loc = coordinate_p_ideal_upscaled[i], size = 1)[0])

        # squash values to sum up to one to represent probabilities
        coordinate_p_noisy_squashed = softmax(coordinate_p_noisy)

        # save group's probability distribution in overall df
        both_groups.append(coordinate_p_noisy_squashed)
        #print(both_groups[group])

    #X.append(both_groups)
    X_coord_conf_noisy.append(both_groups)



100%|██████████| 2000000/2000000 [07:04<00:00, 4710.06it/s]



# HYPERPARAMETER-TUNING: COORDINATE CONFIDENCE NOISE

In [4]:
no_true_inst = 0 # counting how many times an instruction was really given
no_good_pred = 0 # counting right model predictions when really given an instruction

no_two_true_inst = 0 # counting how many times both group1 and group2 really received an instruction
no_two_good_pred = 0 # counting how many times both groups gave the right prediction

max_inst_conf = [] # storing the maximum coordinate confidence per instance for group 1 and group 2
max_coord_conf = [] # storing the maximum instruction confidence per instance for group 1 and group 2


for m in tqdm(range(no_instances)):

    # checking correct predictions per group
    for group in range(2):
        if X_inst_conf_ideal[m][group]==1:
            no_true_inst += 1
            if np.argmax(X_coord_conf_ideal[m])==np.argmax(X_coord_conf_noisy[m][group]):
                no_good_pred += 1

    # checking correct predictions for both groups
    if X_inst_conf_ideal[m][0]==X_inst_conf_ideal[m][1]==1:
        no_two_true_inst +=1

        if np.argmax(X_coord_conf_ideal[m])==np.argmax(X_coord_conf_noisy[m][0])==np.argmax(X_coord_conf_noisy[m][1]):
            no_two_good_pred += 1

    # checking average of highest confidence for both groups
    for group in range(2):
        if X_inst_conf_ideal[m][group]==1:
            max_coord_conf.append(np.max(X_coord_conf_noisy[m][group]))
    for group in range(3):
        if X_inst_conf_ideal[m][group]==1:
            max_inst_conf.append(np.max(X_inst_conf_noisy[m][group]))



print(f'Average correct predictions:  {(no_good_pred*100/no_true_inst):.2f}%')
print(f'Average correct predictions of both models at the same time: {(no_two_good_pred*100/no_two_true_inst):.2f} %')

print(f'Average highest coordinate confidence for both models: {mean(max_coord_conf)}')
print(f'Average highest instruction confidence for both models: {mean(max_inst_conf)}')

100%|██████████| 2000000/2000000 [00:48<00:00, 41349.47it/s]


Average correct predictions:  91.12%
Average correct predictions of both models at the same time: 83.08 %
Average highest coordinate confidence for both models: 0.9321752069066558
Average highest instruction confidence for both models: 0.9523435988892216


In [5]:
no_good_pred_inst = 0

for group in range(3):
    for m in range(no_instances):
        if X_inst_conf_ideal[m][group] == 1 and X_inst_conf_noisy[m][group] > 0.95:
            no_good_pred_inst += 1
        elif X_inst_conf_ideal[m][group] == 0 and X_inst_conf_noisy[m][group] < 0.05:
            no_good_pred_inst += 1

print(f'{no_good_pred_inst/no_instances/3}')

0.6998658333333333


# Gold Label

Legend:  


0 = NaN (no instruction was given by human, therefore no instruction is forwarded to motion group)  

1 = forward language group's output (human gave language-only-instruction, therefore language-only-instruction overrules other instructions)  

2 to (no_puzzlepieces+2) = puzzle piece ID

In [6]:
y = []
for m in tqdm(range(no_instances)):
    
    # language group outputs  
    if X_inst_conf_ideal[m][2]==1:
        y.append(1)
        
    # language and vision group outputs
    elif X_inst_conf_ideal[m][0]==1:
        y.append(np.argmax(X_coord_conf_ideal[m])+2)
        
    # gesture group outputs
    elif X_inst_conf_ideal[m][1]==1:
        y.append(np.argmax(X_coord_conf_ideal[m])+2)
        
    # noone outputs
    else:
        y.append(0)

100%|██████████| 2000000/2000000 [00:02<00:00, 668852.37it/s]


# Store All

In [7]:
dict_y = {'DM output': y}
df_y = pd.DataFrame(data=dict_y)
df_y.to_json(r'y_DM.json', orient='index')

In [8]:
instr_df = pd.DataFrame(X_inst_conf_noisy)
coord_df0 = pd.DataFrame([row[0].tolist() for row in X_coord_conf_noisy])
coord_df1 = pd.DataFrame([row[1].tolist() for row in X_coord_conf_noisy])

a = [f'Instr{i}' for i in range(3)]
b = [f'LV{i}' for i in range(no_puzzlepieces)]
c = [f'G{i}' for i in range(no_puzzlepieces)]

col_names = []
col_names.extend(a)
col_names.extend(b)
col_names.extend(c)

final_df_noisy = pd.concat([instr_df, coord_df0, coord_df1], axis=1)
final_df_noisy.columns = col_names

final_df_noisy.to_json(r'X_DM.json', orient='index')

In [9]:
final_df_noisy.head()

,Instr0,Instr1,Instr2,LV0,LV1,LV2,LV3,LV4,LV5,LV6,...,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14
0,0.044417,0.948645,0.051394,1.184407e-02,0.004668,1.679142e-01,1.223004e-02,6.423568e-02,0.003802,0.075221,...,0.000017,2.453819e-12,5.645929e-10,1.325844e-08,6.800279e-08,2.394640e-07,0.999050,0.000128,1.793379e-08,0.000050
1,0.047812,0.045173,0.952331,2.412665e-02,0.096746,3.777176e-02,2.399562e-01,8.807114e-03,0.098118,0.025521,...,0.196869,2.186837e-02,2.451092e-02,1.230735e-01,5.320467e-03,1.187514e-02,0.009244,0.058429,5.512780e-03,0.087323
2,0.953314,0.956077,0.041540,9.475518e-01,0.002022,1.077363e-06,3.835028e-05,2.452871e-02,0.020271,0.002508,...,0.083525,6.902250e-06,6.435119e-08,8.469432e-08,5.813958e-05,5.263748e-09,0.000087,0.000002,7.223716e-05,0.002428
3,0.050110,0.045864,0.051785,9.889656e-02,0.027604,5.037490e-02,1.142828e-01,8.541672e-02,0.050009,0.007546,...,0.123219,2.595567e-01,1.694002e-01,3.982521e-02,3.949090e-02,2.969280e-02,0.007174,0.009574,3.856792e-02,0.079525
4,0.956507,0.045919,0.041958,8.467011e-08,0.000028,1.567538e-09,5.199476e-07,4.801439e-09,0.000010,0.000010,...,0.009694,4.165330e-02,6.858550e-03,1.709011e-02,3.491676e-01,1.156647e-02,0.026475,0.007702,1.292198e-02,0.043019


In [10]:
# df_new = df.drop(['Instruction Confidence', 'Coordinate Confidence'], axis=1)
# df_new.head()
#
# instr_df = pd.DataFrame(df_new['Instruction'].to_list())
#
# coord_df = pd.DataFrame(df_new['Coordinate'].to_list())
#
# a = [f'Instr{i}' for i in range(3)]
# b = [f'GoldCoord{i}' for i in range(no_puzzlepieces)]
#
# col_names = []
# col_names.extend(a)
# col_names.extend(b)
#
# final_df_gold = pd.concat([instr_df, coord_df], axis=1)
# final_df_gold.columns = col_names
# final_df_gold.head()